# Lecture 3: More Filters!

### Introduction
In this notebook, we'll go over more advanced use cases for filters, including frequency/amplitude filtering and what certain filters do. We'll play around with both from-scratch sounds and imported sounds using our filters. 

In [187]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import IPython

In [66]:
def subplot(a: np.ndarray[float], b: np.ndarray[float]) -> None:
    fig, axs = plt.subplots(1, 2, figsize=(8, 4))
    axs[0].plot(a, color='blue')
    axs[1].plot(b, color='green')


In [ ]:
sr, s = wav.read('percussion.wav') # <- This can be any sound you want
s = (s/np.linalg.norm(s)) * 10

IPython.display.Audio(s, rate=sr, normalize=False)

### Amplitude Modifications

The simplest filter we can make to any sound is modifying its *amplitude* with respect to time. For example, multiplying by a scalar:

In [ ]:
# 0 < x < 1 is quieter,  x > 1 is louder.


What if we want to vary the volume over *time*?

In [ ]:
filter = np.linspace(start = 1.0, stop = 0.0, num=s.shape[0])

What about *fade-ins* and *fade-outs*?

In [ ]:
#print(sr) = 11025 samples/sec

IPython.display.Audio(s * f, rate=sr, normalize=False)

In [ ]:
subplot(s, s * f)

We aren't just limited to linear functions! Any *continuous* function works.

In [ ]:
subplot(s, s * f)

Let's return to our made-from-scratch sound...

In [ ]:
sample_rate = 8000 # how many samples to play per second
t = lambda duration: np.linspace(0, duration, num=int(sample_rate * duration))

sound = lambda f, t, a: 0.1 * a * np.sin(f * 2 * np.pi * t)

original = sound(440, t(3), 1)

IPython.display.Audio(original, rate=sample_rate, normalize=False)

Notice that, alongside the *amplitude*, we use functions or arrays for *frequency* as well! 

In [ ]:
subplot(original, sound_f_a)

In [ ]:
f = 0.7*np.sin(0.5*t(4))
plt.plot(f)
IPython.display.Audio(sound(1000*f, t(4), 0.5), rate=sample_rate, normalize=False)

## Challenge: cross-fading

Suppose we want to fade into our sound from white noise. We want the noise to play for 2 seconds, then fade into our sound for 2 seconds. How would we do that?

In [ ]:
# white noise:
f = 0.05*np.random.normal(size=t(10).shape)
IPython.display.Audio(f, rate=sample_rate, normalize=False)

In [ ]:
# Our sound
sr, s = wav.read('percussion.wav')
s = (s/np.linalg.norm(s)) * 10
s = np.hstack([s, s, s, s])
IPython.display.Audio(s, rate=sr, normalize=False)

In [ ]:
# code goes here...

## Addition: More sinusoids

Sinusoids can *greatly* change the timbre of sound, and can be operated on each other to create new sounds entirely. Check out [Waveforms](https://arc.net/l/quote/oqcqtemn) for more details

In [ ]:
sample_rate = 8000 # how many samples to play per second
t = lambda duration: np.linspace(0, duration, num=int(sample_rate * duration))

sound = lambda f, t, a, sinusoid=np.sin: 0.1 * a * sinusoid(f * 2 * np.pi * t)

original = sound(440, t(3), 1)

IPython.display.Audio(original, rate=sample_rate, normalize=False)

In [ ]:
import scipy
original = sound(440, t(3), 1, scipy.signal.sawtooth)

IPython.display.Audio(original, rate=sample_rate, normalize=False)

In [ ]:
original = sound(440, t(3), 1, scipy.signal.square)

IPython.display.Audio(original, rate=sample_rate, normalize=False)

In [ ]:
f = scipy.signal.triang(t(3).shape[0])

original = sound(440, t(3), 1)

IPython.display.Audio(original, rate=sample_rate, normalize=False)